Comprehensive Classification Problem

Multi class 
Handling class imbalance
logistic regression with hyper parameter tuning
decision tree classifier with hyper parameter tuning
random forest classifier with hyper parameter tuning
SVM Classifier -> linear
SVM Classifer -> kernel
k neighbors classifier with hyper parameter tuning - number of neighbors
Adaboost classifier with hyper-parameter tuning
XGBoost Classifier with hyper-parameter tuning

Note on Bias-Variance trade-off

Difference w.r.t 55. yeast classification - Smote and UnderSampling is that this uses RandomOverSampling only
We have tried RandomOverSampling without shrinkage and with shrinkage.
Note, we have not used both one on top of another like Smote over RandomUndersampling in the other example

In [1]:
import pandas as pd
data = pd.read_csv('dataset_185_yeast.csv')
pd.set_option('display.max_columns',None)
data.head(5)

,mcg,gvh,alm,mit,erl,pox,vac,nuc,class_protein_localization
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


In [2]:
# all the columns are needed
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   mcg                         1484 non-null   float64
 1   gvh                         1484 non-null   float64
 2   alm                         1484 non-null   float64
 3   mit                         1484 non-null   float64
 4   erl                         1484 non-null   float64
 5   pox                         1484 non-null   float64
 6   vac                         1484 non-null   float64
 7   nuc                         1484 non-null   float64
 8   class_protein_localization  1484 non-null   object 
dtypes: float64(8), object(1)
memory usage: 104.5+ KB


In [3]:
data.describe()

,mcg,gvh,alm,mit,erl,pox,vac,nuc
count,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000
mean,0.500121,0.499933,0.500034,0.261186,0.504717,0.007500,0.499885,0.276199
std,0.137299,0.123924,0.086670,0.137098,0.048351,0.075683,0.057797,0.106491
min,0.110000,0.130000,0.210000,0.000000,0.500000,0.000000,0.000000,0.000000
25%,0.410000,0.420000,0.460000,0.170000,0.500000,0.000000,0.480000,0.220000
50%,0.490000,0.490000,0.510000,0.220000,0.500000,0.000000,0.510000,0.220000
75%,0.580000,0.570000,0.550000,0.320000,0.500000,0.000000,0.530000,0.300000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.830000,0.730000,1.000000


In [4]:
# numeric & categorical columns segregation not needed as all are numeric columns
# outlier handling not needed as all columns seem to be of the same scale
# lets look at nulls -> no nulls as well
data.isnull().sum()

mcg                           0
gvh                           0
alm                           0
mit                           0
erl                           0
pox                           0
vac                           0
nuc                           0
class_protein_localization    0
dtype: int64

In [5]:
# class imbalance
data['class_protein_localization'].value_counts()

CYT    463
NUC    429
MIT    244
ME3    163
ME2     51
ME1     44
EXC     35
VAC     30
POX     20
ERL      5
Name: class_protein_localization, dtype: int64

In [6]:
#features and target
features_df = data.drop(['class_protein_localization'], axis = 1)
features_df

# target_df = pd.DataFrame(data['class_protein_localization'])
# target_df

target_df = data['class_protein_localization']

In [7]:
print('Original classes', target_df.value_counts())

Original classes CYT    463
NUC    429
MIT    244
ME3    163
ME2     51
ME1     44
EXC     35
VAC     30
POX     20
ERL      5
Name: class_protein_localization, dtype: int64


In [7]:
# ! pip install imblearn

In [8]:
# without using shrinkage

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state = 42)
features_df_ros, target_dt_ros = ros.fit_resample(features_df, target_df)

In [9]:
print('After handling class imbalance: ', target_dt_ros.value_counts())

After handling class imbalance:  NUC    463
ME2    463
MIT    463
EXC    463
ME3    463
POX    463
ERL    463
CYT    463
VAC    463
ME1    463
Name: class_protein_localization, dtype: int64


In [10]:
print(features_df.shape)
print(target_df.shape)
print(features_df_ros.shape)
print(target_dt_ros.shape)

(1484, 8)
(1484,)
(4630, 8)
(4630,)


In [15]:
# i looked at the 2 csv files for ERL that has only 5 values in the original dataset
# i took one record out of these 5 i.e. mcg = 0.86 and found that in the over-sampled data, there are 109 exact records for this
# by exact i mean all the other attributes had the same values as the original dataset

original_data = features_df.join(target_df)
original_data.to_csv('original_data.csv')
over_sampled_data = features_df_ros.join(target_dt_ros)
over_sampled_data.to_csv('over_sampled_data.csv')

In [16]:
# using shrinkage factor

ros = RandomOverSampler(random_state = 42, shrinkage = 3)
features_df_ros, target_dt_ros = ros.fit_resample(features_df, target_df)

In [17]:
print('After handling class imbalance with shrinkage factor : ', target_dt_ros.value_counts())

After handling class imbalance with shrinkage factor :  NUC    463
ME2    463
MIT    463
EXC    463
ME3    463
POX    463
ERL    463
CYT    463
VAC    463
ME1    463
Name: class_protein_localization, dtype: int64


In [18]:
# Now i see the newly added rows are not exact copies

over_sampled_data_with_shrinkage = features_df_ros.join(target_dt_ros)
over_sampled_data_with_shrinkage.to_csv('over_sampled_data_with_shrinkage.csv')

In [37]:
# Smote did not work for me

# from imblearn.over_sampling import SMOTE
# smote = SMOTE(k_neighbors=3)
# features_df_smote, target_dt_smote = smote.fit_resample(features_df, target_df)

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_df_ros, target_dt_ros, train_size = 0.8, test_size = 0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3704, 8)
(926, 8)
(3704,)
(926,)


In [39]:
# naive logistic regression

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 2)
lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

C:\Users\Vinod\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [40]:
y_train_pred
y_test_pred

array(['POX', 'MIT', 'ME2', 'ME1', 'ERL', 'ME3', 'ME1', 'ME1', 'EXC',
       'MIT', 'EXC', 'NUC', 'ME3', 'ERL', 'ME1', 'CYT', 'ME3', 'ME3',
       'NUC', 'EXC', 'POX', 'VAC', 'ERL', 'EXC', 'ME3', 'MIT', 'ME1',
       'VAC', 'MIT', 'ME1', 'MIT', 'NUC', 'EXC', 'ME2', 'CYT', 'EXC',
       'CYT', 'ERL', 'VAC', 'MIT', 'EXC', 'ME2', 'ME3', 'CYT', 'EXC',
       'ME3', 'CYT', 'MIT', 'EXC', 'ME2', 'ERL', 'NUC', 'ME3', 'ME1',
       'CYT', 'ERL', 'EXC', 'POX', 'NUC', 'ERL', 'ME1', 'CYT', 'ME3',
       'EXC', 'VAC', 'MIT', 'CYT', 'POX', 'CYT', 'MIT', 'POX', 'ME2',
       'ME3', 'ME1', 'ME1', 'ERL', 'ERL', 'ME3', 'NUC', 'VAC', 'MIT',
       'ME2', 'POX', 'EXC', 'MIT', 'ERL', 'MIT', 'CYT', 'ME2', 'ME2',
       'ERL', 'ERL', 'EXC', 'POX', 'ME3', 'ME3', 'ME2', 'VAC', 'POX',
       'MIT', 'ME2', 'MIT', 'NUC', 'ME1', 'CYT', 'MIT', 'EXC', 'MIT',
       'ERL', 'VAC', 'EXC', 'ME3', 'EXC', 'POX', 'EXC', 'EXC', 'ERL',
       'ERL', 'ERL', 'ME3', 'ME2', 'ERL', 'NUC', 'ME1', 'CYT', 'EXC',
       'POX', 'CYT',

In [41]:
import sklearn.metrics as metrics

accuracy = metrics.accuracy_score(y_test, y_test_pred)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')

# Without average = weighted, you will get below error
# ValueError: Target is multiclass but average='binary'. 
# Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

print('accuracy: ', accuracy)
print('error: ', error)
print('precision: ', precision)
print('recall: ', recall)
print('f1 score: ', f1_score)

cm = metrics.confusion_matrix(y_test, y_test_pred)
print('The confusion matrix is: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING LOGISTIC REGRESSION *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.5086393088552916
error:  0.4913606911447084
precision:  0.5016456669010374
recall:  0.5086393088552916
f1 score:  0.4962635852883084
The confusion matrix is: 
[[ 44   0   2   0   0   3  12  14   0  18]
 [  0 115   0   0   0   0   0   0   0   0]
 [  8   0  38  16   8   2   6   3   0   5]
 [  0   0  25  60   9   2   1   1   0   4]
 [  2   3  19  16  18  18  10   1   0   4]
 [  1   1   0   4   1  59   5   4   0   7]
 [ 18   0   7   3   2   5  41   7   6   7]
 [ 26   0   1   0   1   4   3  38   0   3]
 [  5   0   4   5   3   6   5   0  52   3]
 [ 21   0  13   6  10  28   7  11   0   6]]


***************** CLASSIFICATION REPORT USING LOGISTIC REGRESSION *******************
              precision    recall  f1-score   support

         CYT       0.35      0.47      0.40        93
         ERL       0.97      1.00      0.98       115
         EXC       0.35      0.44      0.39        86
         ME1       0.55      0.59      0.57       102
         ME2       0.35      0.20     

In [42]:
# hyper parameter tuning for logistic regression

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

base_lr = LogisticRegression(random_state = 2)

params = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'dual': [True, False],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

cv_lr = GridSearchCV(base_lr, cv = 10, param_grid = params, n_jobs = 3)
cv_lr.fit(X_train, y_train)
print('Best parameters using Grid Search are: ', cv_lr.best_params_)

Best parameters using Grid Search are:  {'dual': True, 'penalty': 'l2', 'solver': 'liblinear'}


C:\Users\Vinod\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
280 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Vinod\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Vinod\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Vinod\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 61, in _check_solver
    raise ValueError(
Val

In [43]:
base_lr.set_params(penalty = 'l2', dual = True, solver = 'liblinear')
base_lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

In [44]:
# no improvement

import sklearn.metrics as metrics

accuracy = metrics.accuracy_score(y_test, y_test_pred)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')

# Without average = weighted, you will get below error
# ValueError: Target is multiclass but average='binary'. 
# Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

print('accuracy: ', accuracy)
print('error: ', error)
print('precision: ', precision)
print('recall: ', recall)
print('f1 score: ', f1_score)

cm = metrics.confusion_matrix(y_test, y_test_pred)
print('The confusion matrix is: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING TUNED LOGISTIC REGRESSION *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.5086393088552916
error:  0.4913606911447084
precision:  0.5016456669010374
recall:  0.5086393088552916
f1 score:  0.4962635852883084
The confusion matrix is: 
[[ 44   0   2   0   0   3  12  14   0  18]
 [  0 115   0   0   0   0   0   0   0   0]
 [  8   0  38  16   8   2   6   3   0   5]
 [  0   0  25  60   9   2   1   1   0   4]
 [  2   3  19  16  18  18  10   1   0   4]
 [  1   1   0   4   1  59   5   4   0   7]
 [ 18   0   7   3   2   5  41   7   6   7]
 [ 26   0   1   0   1   4   3  38   0   3]
 [  5   0   4   5   3   6   5   0  52   3]
 [ 21   0  13   6  10  28   7  11   0   6]]


***************** CLASSIFICATION REPORT USING TUNED LOGISTIC REGRESSION *******************
              precision    recall  f1-score   support

         CYT       0.35      0.47      0.40        93
         ERL       0.97      1.00      0.98       115
         EXC       0.35      0.44      0.39        86
         ME1       0.55      0.59      0.57       102
         ME2       0.35      0.2

In [45]:
# support vector - linear
# no improvement over logistic regression

from sklearn.svm import SVC
model_svc = SVC(kernel = 'linear', random_state = 0)
model_svc.fit(X_train, y_train)
y_train_pred = model_svc.predict(X_train)
y_test_pred = model_svc.predict(X_test)

In [46]:
import sklearn.metrics as metrics

accuracy = metrics.accuracy_score(y_test, y_test_pred)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')
cm = metrics.confusion_matrix(y_test, y_test_pred)

print('accuracy: ', accuracy)
print('error: ', error)
print('precision: ', precision)
print('recall: ', recall)
print('f1 score: ', f1_score)

print('cm: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING TUNED SVM LINEAR CLASSIFIER *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.4956803455723542
error:  0.5043196544276458
precision:  0.508254962246054
recall:  0.4956803455723542
f1 score:  0.4757581041405796
cm: 
[[ 67   0   2   0   0   2   8   9   0   5]
 [  0 115   0   0   0   0   0   0   0   0]
 [ 11   0  38  16   6   2   6   2   0   5]
 [  1   0  26  62   6   2   1   0   0   4]
 [  5   2  15  20  16  18   6   1   0   8]
 [  5   1   0   4   2  60   3   2   0   5]
 [ 27   0   6   4   3   5  38   5   2   6]
 [ 43   1   1   0   1   1   3  24   0   2]
 [ 14   0   8   5   4   7   6   2  34   3]
 [ 29   0  13   7   9  30   4   5   0   5]]


***************** CLASSIFICATION REPORT USING TUNED SVM LINEAR CLASSIFIER *******************
              precision    recall  f1-score   support

         CYT       0.33      0.72      0.45        93
         ERL       0.97      1.00      0.98       115
         EXC       0.35      0.44      0.39        86
         ME1       0.53      0.61      0.56       102
         ME2       0.34      0.18      0.23        9

In [47]:
# support vector - rbf
# small improvement found

from sklearn.svm import SVC
model_svc = SVC(kernel = 'rbf', random_state = 0)
model_svc.fit(X_train, y_train)
y_train_pred = model_svc.predict(X_train)
y_test_pred = model_svc.predict(X_test)

In [48]:
import sklearn.metrics as metrics

accuracy = metrics.accuracy_score(y_test, y_test_pred)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')
cm = metrics.confusion_matrix(y_test, y_test_pred)

print('accuracy: ', accuracy)
print('error: ', error)
print('precision: ', precision)
print('recall: ', recall)
print('f1 score: ', f1_score)

print('cm: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING TUNED SVM rbf CLASSIFIER *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.5809935205183585
error:  0.4190064794816415
precision:  0.5922150003123872
recall:  0.5809935205183585
f1 score:  0.5629560410363366
cm: 
[[ 70   0   1   0   0   5   8   8   0   1]
 [  0 115   0   0   0   0   0   0   0   0]
 [ 10   0  51  11   2   3   3   1   0   5]
 [  0   0  20  74   3   1   2   1   0   1]
 [  6   2  17  17  16  17   6   1   0   9]
 [  3   1   0   5   2  60   2   2   0   7]
 [ 25   0   6   7   1   7  37   4   2   7]
 [ 37   0   2   0   1   1   4  29   0   2]
 [  4   0   2   3   2   2   1   0  66   3]
 [ 20   0  12   8   4  27   4   7   0  20]]


***************** CLASSIFICATION REPORT USING TUNED SVM rbf CLASSIFIER *******************
              precision    recall  f1-score   support

         CYT       0.40      0.75      0.52        93
         ERL       0.97      1.00      0.99       115
         EXC       0.46      0.59      0.52        86
         ME1       0.59      0.73      0.65       102
         ME2       0.52      0.18      0.26        91


In [49]:
# good improvement found -> this improvement only after handling class imbalance

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

base_dt = DecisionTreeClassifier(random_state = 10)

params = {
    'criterion': ['gini','entropy'],
    'splitter':['random','best'],
    'max_features':['auto','sqrt','log2',None]
#     'max_depth':[15,30,45,None],
#     'min_samples_split':[2,5],
#     'min_samples_leaf':[1,3,5]
}

cv_dt = GridSearchCV(base_dt, cv = 10, param_grid = params, n_jobs = 3)

cv_dt.fit(X_train, y_train)

print('Best Parameters using Grid search: \n', cv_dt.best_params_)

Best Parameters using Grid search: 
 {'criterion': 'entropy', 'max_features': None, 'splitter': 'best'}


In [50]:
base_dt.set_params(criterion = 'entropy', max_features = None, splitter = 'best', random_state = 10)
base_dt.fit(X_train, y_train)
y_train_pred = base_dt.predict(X_train)
y_test_pred = base_dt.predict(X_test)

In [51]:
import sklearn.metrics as metrics

accuracy = metrics.accuracy_score(y_test, y_test_pred)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')
cm = metrics.confusion_matrix(y_test, y_test_pred)

print('accuracy: ', accuracy)
print('error: ', error)
print('precision: ', precision)
print('recall: ', recall)
print('f1_score: ', f1_score)

print('\n')
print('The confusion matrix is: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING TUNED DECISION TREE CLASSIFIER *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.6274298056155507
error:  0.37257019438444927
precision:  0.6288442832861935
recall:  0.6274298056155507
f1_score:  0.62685349001702


The confusion matrix is: 
[[ 38   0   1   0   1   6  16  25   1   5]
 [  0 114   0   0   0   0   0   1   0   0]
 [  6   0  41   9   4   1   5   1   2  17]
 [  1   0  13  67   3   1   2   1   0  14]
 [  3   0   4   0  61  17   0   5   0   1]
 [  2   0   1   0  19  54   0   2   0   4]
 [  9   0   2   1   0   4  52   9  15   4]
 [ 26   0   3   0   3   1   7  34   1   1]
 [  0   0   0   0   0   0   9   0  74   0]
 [  6   0  14  12   2   9   2  11   0  46]]


***************** CLASSIFICATION REPORT USING TUNED DECISION TREE CLASSIFIER *******************
              precision    recall  f1-score   support

         CYT       0.42      0.41      0.41        93
         ERL       1.00      0.99      1.00       115
         EXC       0.52      0.48      0.50        86
         ME1       0.75      0.66      0.70       102
         ME2       0.66   

In [59]:
# Good improvement

from sklearn.ensemble import RandomForestClassifier
base_rf = RandomForestClassifier()
base_rf.fit(X_train, y_train)
y_train_pred = base_rf.predict(X_train)
y_test_pred = base_rf.predict(X_test)

In [60]:
param_dist = {
             'n_estimators': [100,140,180],
             'criterion':['gini','entropy','log_loss'],
             'max_features':['auto','sqrt','log2',None]
#            'max_depth':[2,3,4,5,6,7,8],
#            'min_samples_split':[2,3,4,5,6],
#            'min_samples_leaf':[2,3,4,5,6]
               }

base_rf = RandomForestClassifier(random_state = 10)
cv_rf = GridSearchCV(base_rf, cv=10, param_grid=param_dist, n_jobs=3)
cv_rf.fit(X_train, y_train)

print('Best parameters using Grid Search are: ', cv_rf.best_params_)

Best parameters using Grid Search are:  {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 180}


In [61]:
base_rf.set_params(criterion = 'entropy', max_features = 'log2', n_estimators = 180)
base_rf.fit(X_train, y_train)
y_train_pred = base_rf.predict(X_train)
y_test_pred = base_rf.predict(X_test)


In [58]:
# best so far

import sklearn.metrics as metrics

accuracy = metrics.accuracy_score(y_test, y_test_pred)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')
cm = metrics.confusion_matrix(y_test, y_test_pred)

print('accuracy: ', accuracy)
print('error: ', error)
print('precision: ', precision)
print('recall: ', recall)
print('f1_score: ', f1_score)

print('\n')
print('The confusion matrix is: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING TUNED RANDOM FOREST CLASSIFIER *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.7181425485961123
error:  0.28185745140388774
precision:  0.7230298624383488
recall:  0.7181425485961123
f1_score:  0.7154930340087589


The confusion matrix is: 
[[ 55   0   4   0   0   3   9  22   0   0]
 [  0 115   0   0   0   0   0   0   0   0]
 [  3   0  58  13   0   0   2   0   0  10]
 [  0   0  12  82   0   1   0   1   0   6]
 [  1   0   3   3  65  16   0   3   0   0]
 [  0   0   0   0  12  66   0   1   0   3]
 [ 15   0   2   1   1   3  50   4  17   3]
 [ 23   0   0   0   2   2   0  44   0   5]
 [  0   0   0   0   0   0   5   0  78   0]
 [  4   0  10  13   1   9   0  13   0  52]]


***************** CLASSIFICATION REPORT USING TUNED RANDOM FOREST CLASSIFIER *******************
              precision    recall  f1-score   support

         CYT       0.54      0.59      0.57        93
         ERL       1.00      1.00      1.00       115
         EXC       0.65      0.67      0.66        86
         ME1       0.73      0.80      0.77       102
         ME2       0.80 

In [28]:
import pickle

with open('C:\\Vinod\\Machine_Learning_Deployment\\models\\yeast_classification_rf_model.pkl','wb') as yeast_classification_rf_model_pkl_file:
    pickle.dump(base_rf, yeast_classification_rf_model_pkl_file)

In [62]:
# trying out KNN also but RF is the best

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10) # 10 because this is the number of classes
knn.fit(X_train, y_train)

y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

import sklearn.metrics as metrics
cm = metrics.confusion_matrix(y_test, y_test_pred)
accuracy = metrics.accuracy_score(y_test, y_test_pred, normalize = True)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')

print('accuracy: ', accuracy)
print('precision: ', precision)
print('error: ', error)
print('recall: ', recall)
print('f1 score: ', f1_score)

print('\n')
print('The confusion matrix is: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING KNeigbors CLASSIFIER *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.5637149028077754
precision:  0.5746810864078601
error:  0.43628509719222464
recall:  0.5637149028077754
f1 score:  0.5511498572880486


The confusion matrix is: 
[[ 59   0   1   0   0   2   9  21   0   1]
 [  0 115   0   0   0   0   0   0   0   0]
 [ 12   0  48  10   3   2   3   4   0   4]
 [  0   0  18  68   6   3   1   1   0   5]
 [  6   1  18  14  23  15   6   3   0   5]
 [  6   1   4   4   3  53   2   2   0   7]
 [ 20   0  12   6   1   6  37   7   5   2]
 [ 34   0   1   0   1   1   3  36   0   0]
 [  4   0   2   2   1   3   1   1  69   0]
 [ 24   0  17   6   8  20   3  10   0  14]]


***************** CLASSIFICATION REPORT USING KNeigbors CLASSIFIER *******************
              precision    recall  f1-score   support

         CYT       0.36      0.63      0.46        93
         ERL       0.98      1.00      0.99       115
         EXC       0.40      0.56      0.46        86
         ME1       0.62      0.67      0.64       102
         ME2       0.50      0.25 

In [63]:
k_range = range(1, 31)
accuracy_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train.values.ravel())
    y_train_pred = knn.predict(X_train)
    y_test_pred = knn.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_test_pred, normalize = True)
    accuracy_scores.append(accuracy)
    print('number of neighbors: ', k, ' accuracy: ', accuracy)    

number of neighbors:  1  accuracy:  0.5226781857451404
number of neighbors:  2  accuracy:  0.5485961123110151
number of neighbors:  3  accuracy:  0.5453563714902808
number of neighbors:  4  accuracy:  0.5475161987041036
number of neighbors:  5  accuracy:  0.5485961123110151
number of neighbors:  6  accuracy:  0.5550755939524838
number of neighbors:  7  accuracy:  0.5572354211663066
number of neighbors:  8  accuracy:  0.5615550755939525
number of neighbors:  9  accuracy:  0.5572354211663066
number of neighbors:  10  accuracy:  0.5637149028077754
number of neighbors:  11  accuracy:  0.5701943844492441
number of neighbors:  12  accuracy:  0.5680345572354212
number of neighbors:  13  accuracy:  0.5701943844492441
number of neighbors:  14  accuracy:  0.5593952483801296
number of neighbors:  15  accuracy:  0.5561555075593952
number of neighbors:  16  accuracy:  0.562634989200864
number of neighbors:  17  accuracy:  0.5615550755939525
number of neighbors:  18  accuracy:  0.5561555075593952
nu

In [64]:
# n_neigbors = 1 not advisable because such a model has high variance
# see notes below

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 28) 
knn.fit(X_train, y_train)

y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

import sklearn.metrics as metrics
cm = metrics.confusion_matrix(y_test, y_test_pred)
accuracy = metrics.accuracy_score(y_test, y_test_pred, normalize = True)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')

print('accuracy: ', accuracy)
print('precision: ', precision)
print('error: ', error)
print('recall: ', recall)
print('f1 score: ', f1_score)

print('\n')
print('The confusion matrix is: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING KNeighbors CLASSIFIER *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.5712742980561555
precision:  0.5823414122115358
error:  0.4287257019438445
recall:  0.5712742980561555
f1 score:  0.5465053360077358


The confusion matrix is: 
[[ 70   0   2   0   0   0   6  15   0   0]
 [  0 115   0   0   0   0   0   0   0   0]
 [ 14   0  51   9   2   2   3   3   0   2]
 [  1   0  22  72   1   2   1   1   0   2]
 [ 10   2  19  16  15  17   5   1   0   6]
 [  6   1   0   3   1  61   3   4   0   3]
 [ 23   0   9   2   1   7  42   6   3   3]
 [ 40   0   0   0   1   0   3  31   0   1]
 [  5   0   4   2   1   3   1   2  65   0]
 [ 25   0  14   9   6  27   2  12   0   7]]


***************** CLASSIFICATION REPORT USING KNeighbors CLASSIFIER *******************
              precision    recall  f1-score   support

         CYT       0.36      0.75      0.49        93
         ERL       0.97      1.00      0.99       115
         EXC       0.42      0.59      0.49        86
         ME1       0.64      0.71      0.67       102
         ME2       0.54      0.16 

In general, a k-NN model fits a specific point in the data with the N nearest data points in your training set. 
For 1-NN this point depends only of 1 single other point. 
E.g. you want to split your samples into two groups (classification) - red and blue. 
If you train your model for a certain point p for which the nearest 4 neighbors would be red, blue, blue, blue (ascending by distance to p). 
Then a 4-NN would classify your point to blue (3 times blue and 1 time red), but your 1-NN model classifies it to red, because red is the nearest point. 
This means, that your model is really close to your training data and therefore the bias is low. 
If you compute the RSS between your model and your training data it is close to 0. 
In contrast to this the variance in your model is high, because your model is extremely sensitive and wiggly. 
As pointed out above, a random shuffling of your training set would be likely to change your model dramatically. 
In contrast, 10-NN would be more robust in such cases, but could be to stiff. Which k to choose depends on your data set. 
This highly depends on the Bias-Variance-Tradeoff, which exactly relates to this problem.

What is bias?
Model with high bias pays very little attention to the training data and oversimplifies the model. 
It always leads to high error on training and test data.

What is variance?
Model with high variance pays a lot of attention to training data and does not generalize on the data which it hasn’t seen before. 
As a result, such models perform very well on training data but has high error rates on test data.

Low bias/ Low variance is good


In [65]:
from sklearn.ensemble import AdaBoostClassifier

param_dist = {
             'base_estimator':[DecisionTreeClassifier(), RandomForestClassifier()],
             'n_estimators':[70,120,180,20],
             'algorithm':['SAMME','SAMME.R']
             }

model_adaboost = AdaBoostClassifier(random_state = 10)
cv_model_adaboost = GridSearchCV(model_adaboost, cv = 20, param_grid = param_dist, n_jobs=3)
cv_model_adaboost.fit(X_train,y_train)
print('Best Parameters using Grid search: \n', cv_model_adaboost.best_params_)

Best Parameters using Grid search: 
 {'algorithm': 'SAMME', 'base_estimator': RandomForestClassifier(), 'n_estimators': 70}


In [66]:
model_adaboost.set_params(base_estimator = RandomForestClassifier(), n_estimators = 70, algorithm = 'SAMME')
model_adaboost.fit(X_train, y_train)
y_train_pred = model_adaboost.predict(X_train)
y_test_pred = model_adaboost.predict(X_test)

import sklearn.metrics as metrics
cm = metrics.confusion_matrix(y_test, y_test_pred)
accuracy = metrics.accuracy_score(y_test, y_test_pred, normalize = True)
error = 1 - accuracy
precision = metrics.precision_score(y_test, y_test_pred, average = 'weighted')
recall = metrics.recall_score(y_test, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(y_test, y_test_pred, average = 'weighted')

print('accuracy: ', accuracy)
print('precision: ', precision)
print('error: ', error)
print('recall: ', recall)
print('f1 score: ', f1_score)

print('\n')
print('The confusion matrix is: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING ADABOOST CLASSIFIER *******************')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

accuracy:  0.712742980561555
precision:  0.71895042539007
error:  0.28725701943844495
recall:  0.712742980561555
f1 score:  0.7096014150122245


The confusion matrix is: 
[[ 57   0   3   0   0   1   9  21   0   2]
 [  0 115   0   0   0   0   0   0   0   0]
 [  4   0  57  11   0   1   2   2   0   9]
 [  0   0  15  80   0   2   0   1   0   4]
 [  1   0   5   2  65  16   0   1   0   1]
 [  0   1   0   0  10  69   0   1   0   1]
 [ 11   0   2   2   0   4  47   6  19   5]
 [ 26   0   1   0   2   2   1  42   0   2]
 [  0   0   0   1   0   0   5   0  77   0]
 [  8   0  11  10   1  10   1  10   0  51]]


***************** CLASSIFICATION REPORT USING ADABOOST CLASSIFIER *******************
              precision    recall  f1-score   support

         CYT       0.53      0.61      0.57        93
         ERL       0.99      1.00      1.00       115
         EXC       0.61      0.66      0.63        86
         ME1       0.75      0.78      0.77       102
         ME2       0.83      0.71      

In [67]:
y_train.value_counts()

NUC    387
ME3    381
POX    380
EXC    377
ME2    372
CYT    370
MIT    367
VAC    361
ME1    361
ERL    348
Name: class_protein_localization, dtype: int64

In [68]:
# XGBoost does not seem to work on categorized target

import numpy as np
y_train_df = pd.DataFrame(y_train)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_df['indexed'] = le.fit_transform(y_train_df)
y_train_df.head(5)
new_y_train_df = y_train_df['indexed']
new_y_train_df.head(5)

C:\Users\Vinod\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2748    3
1177    0
3278    5
2779    3
657     7
Name: indexed, dtype: int32

In [77]:
# import xgboost as xgb

# param_dist = {
#              'booster':['gbtree', 'gblinear','dart'],
#              'eta': [0.3,0.1],
#              'gamma': [0,5,10],
#              'max_depth':[6,12,18,25],
#              'min_child_weight':[0,1,4,8],
#              'subsample':[0,0.5,1],
#              'tree_method':['auto', 'exact', 'approx', 'hist', 'gpu_hist']
#              }

# model_xgboost = xgb.XGBClassifier(random_state = 10)
# cv_model_xgboost = GridSearchCV(model_xgboost, cv = 20, param_grid = param_dist, n_jobs=3)
# cv_model_xgboost.fit(X_train,new_y_train_df)
# print('Best Parameters using Grid search: \n', cv_model_xgboost.best_params_)

In [78]:
model_xgboost.set_params(max_depth = 25, min_child_weight = 1, tree_method = 'approx')
model_xgboost.fit(X_train, new_y_train_df)
y_train_pred = model_xgboost.predict(X_train)
y_test_pred = model_xgboost.predict(X_test)

In [79]:
y_test_df = pd.DataFrame(y_test)
y_test_df['indexed'] = le.fit_transform(y_test_df)
new_y_test_df = y_test_df['indexed']

C:\Users\Vinod\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [80]:
# xgboost gives better results than adaboost, RF

import sklearn.metrics as metrics
cm = metrics.confusion_matrix(new_y_test_df, y_test_pred)
accuracy = metrics.accuracy_score(new_y_test_df, y_test_pred, normalize = True)
error = 1 - accuracy
precision = metrics.precision_score(new_y_test_df, y_test_pred, average = 'weighted')
recall = metrics.recall_score(new_y_test_df, y_test_pred, average = 'weighted')
f1_score = metrics.f1_score(new_y_test_df, y_test_pred, average = 'weighted')

print('accuracy: ', accuracy)
print('precision: ', precision)
print('error: ', error)
print('recall: ', recall)
print('f1 score: ', f1_score)

print('\n')
print('The confusion matrix is: ')
print(cm)

print('\n')
print('***************** CLASSIFICATION REPORT USING XGBOOST CLASSIFIER *******************')

from sklearn.metrics import classification_report
print(classification_report(new_y_test_df, y_test_pred))

accuracy:  0.7300215982721382
precision:  0.7312437497618721
error:  0.2699784017278618
recall:  0.7300215982721382
f1 score:  0.7283563304295323


The confusion matrix is: 
[[ 52   0   0   0   0   2  13  24   0   2]
 [  0 115   0   0   0   0   0   0   0   0]
 [  3   0  60  10   0   0   2   0   0  11]
 [  0   0  13  80   0   1   0   1   0   7]
 [  0   0   1   4  70  14   0   2   0   0]
 [  1   0   0   0  13  67   0   0   0   1]
 [ 15   0   1   2   0   3  53   5  16   1]
 [ 24   0   0   0   2   2   3  42   0   3]
 [  0   0   0   0   0   0   6   0  77   0]
 [  3   0  13   9   1   8   0   8   0  60]]


***************** CLASSIFICATION REPORT USING XGBOOST CLASSIFIER *******************
              precision    recall  f1-score   support

           0       0.53      0.56      0.54        93
           1       1.00      1.00      1.00       115
           2       0.68      0.70      0.69        86
           3       0.76      0.78      0.77       102
           4       0.81      0.77    